In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

import pandas as pd
import numpy as np
import time

In [7]:
ser = Service('../chromedriver/chromedriver.exe')
driver = webdriver.Chrome(service = ser)

### 01 starbucks crawling를 통한 데이터 목록 생성

In [9]:
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [16]:
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, seoul_btn).click()

In [17]:
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, all_btn).click()
time.sleep(3)

In [21]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [101]:
sb_soup_list = soup.select('li.quickResultLstCon')
sb = sb_soup_list[0]

In [124]:
name = sb['data-name']
lat = sb['data-lat']
long = sb['data-long']
store_type = sb.select('i')[0]['class'][0][4:]
address = str(sb.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
tel = str(sb.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
print(name, lat, long, store_type, address, tel, sep='\n')

역삼아레나빌딩
37.501087
127.043069
general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [125]:
sb_list = []
sb_list_df = pd.DataFrame()

for item in sb_soup_list:
    name = item['data-name']
    lat = item['data-lat']
    long = item['data-long']
    store_type = item.select('i')[0]['class'][0][4:]
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    sb_list.append([name, lat, long, store_type, address, tel])
    sb_list_df = sb_list_df.append(sb_list)
sb_list_df.columns = ['name', 'lat', 'long', 'store_type', 'address', 'tel']
sb_list_df.head()

,name,lat,long,store_type,address,tel
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232


##### 엑셀로 저장하기

In [126]:
sb_list_df.to_excel('./files/seoul_starbucks_class.xlsx', index = False)

### 02 서울열린데이터광장의 OPEN API를 활용한 공공데이터 수집

In [127]:
import requests

In [133]:
SEOUL_API_AUTH_KEY = '자신의 API번호를 입력하세요.'
Open_API = 'GangseoListLoanCompany'

url = 'http://openapi.seoul.go.kr:8088/{}/json/{}/1/5/'.format(SEOUL_API_AUTH_KEY,Open_API)

In [137]:
result_dict = requests.get(url).json()

In [146]:
result_dict['GangseoListLoanCompany']['RESULT']['CODE']      # INFO-000'가 아닐 경우 서버 접속에 error 발생

'INFO-000'

In [151]:
result_dict['GangseoListLoanCompany']['row'][0]['WRKR_RD_FULL_ADDR'].strip()

'서울특별시 강서구 공항대로41길 51, 세신그린코아빌딩 9층 905호 (등촌동)'

In [153]:
smaple_df = pd.DataFrame(result_dict['GangseoListLoanCompany']['row'])

In [156]:
SEOUL_API_AUTH_KEY = '자신의 API번호를 입력하세요.'

def seoul_open_api_data(url, service):
    data_list = None
    try:
        result_dict = requests.get(url).json()
        result_data = result_dict[service]
        code=result_data['RESULT']['CODE']
        if code =='INFO-000':
            data_list = result_data['row']
    except:
        pass
    
    return data_list

##### 서울시 시군구별 인구 데이터 가져오기

In [3]:
sgg_pop_df = pd.read_csv('./files/report.txt', sep='\t', header=2)

sgg_pop_df.head()

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2020.3/4,합계,"4,405,833","9,953,009","4,840,912","5,112,097","9,699,232","4,719,170","4,980,062","253,777","121,742","132,035",2.20,"1,552,356"
1,2020.3/4,종로구,"74,861","159,842","77,391","82,451","149,952","73,024","76,928","9,890","4,367","5,523",2.00,"28,396"
2,2020.3/4,중구,"63,594","135,321","66,193","69,128","125,800","61,526","64,274","9,521","4,667","4,854",1.98,"24,265"
3,2020.3/4,용산구,"112,451","244,953","119,074","125,879","229,786","110,604","119,182","15,167","8,470","6,697",2.04,"39,995"
4,2020.3/4,성동구,"136,096","302,695","147,582","155,113","295,591","144,444","151,147","7,104","3,138","3,966",2.17,"45,372"


In [161]:
sgg_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  26 non-null     object 
 1   JACHIGU                26 non-null     object 
 2   SEDAE                  26 non-null     object 
 3   GYE_1                  26 non-null     object 
 4   NAMJA_1                26 non-null     object 
 5   YEOJA_1                26 non-null     object 
 6   GYE_2                  26 non-null     object 
 7   NAMJA_2                26 non-null     object 
 8   YEOJA_2                26 non-null     object 
 9   GYE_3                  26 non-null     object 
 10  NAMJA_3                26 non-null     object 
 11  YEOJA_3                26 non-null     object 
 12  SEDAEDANGINGU          26 non-null     float64
 13  N_65SEISANGGORYEONGJA  26 non-null     object 
dtypes: float64(1), object(13)
memory usage: 3.0+ KB


In [4]:
sgg_pop_df['자치구']       # 합계 불필요

0       합계
1      종로구
2       중구
3      용산구
4      성동구
5      광진구
6     동대문구
7      중랑구
8      성북구
9      강북구
10     도봉구
11     노원구
12     은평구
13    서대문구
14     마포구
15     양천구
16     강서구
17     구로구
18     금천구
19    영등포구
20     동작구
21     관악구
22     서초구
23     강남구
24     송파구
25     강동구
Name: 자치구, dtype: object

##### 합계  데이터  지우기

In [5]:
condition = sgg_pop_df['자치구'] != '합계'
sgg_pop_df_select = sgg_pop_df[condition]
sgg_pop_df_select.head()

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
1,2020.3/4,종로구,"74,861","159,842","77,391","82,451","149,952","73,024","76,928","9,890","4,367","5,523",2.00,"28,396"
2,2020.3/4,중구,"63,594","135,321","66,193","69,128","125,800","61,526","64,274","9,521","4,667","4,854",1.98,"24,265"
3,2020.3/4,용산구,"112,451","244,953","119,074","125,879","229,786","110,604","119,182","15,167","8,470","6,697",2.04,"39,995"
4,2020.3/4,성동구,"136,096","302,695","147,582","155,113","295,591","144,444","151,147","7,104","3,138","3,966",2.17,"45,372"
5,2020.3/4,광진구,"166,857","361,923","174,077","187,846","348,064","168,095","179,969","13,859","5,982","7,877",2.09,"50,047"


In [6]:
columns = ['자치구', '계']
sgg_pop_df_final = sgg_pop_df_select[columns]
sgg_pop_df_final.head()

,자치구,계
1,종로구,"159,842"
2,중구,"135,321"
3,용산구,"244,953"
4,성동구,"302,695"
5,광진구,"361,923"


In [7]:
sgg_pop_df_final.columns=['시군구명', '주민등록인구']
sgg_pop_df_final.head()

,시군구명,주민등록인구
1,종로구,"159,842"
2,중구,"135,321"
3,용산구,"244,953"
4,성동구,"302,695"
5,광진구,"361,923"


In [8]:
sgg_biz_df = pd.read_csv('./files/report2.txt', sep='\t', header =2)
sgg_biz_df.head()

,기간,자치구,동,사업체수,여성대표자,계,남,여,사업체수.1,종사자수,...,사업체수.15,종사자수.14,사업체수.16,종사자수.15,사업체수.17,종사자수.16,사업체수.18,종사자수.17,사업체수.19,종사자수.18
0,2018,합계,합계,"823,385","279,711","5,210,936","2,878,227","2,332,709",30,462,...,"1,282","140,121","35,377","346,219","28,824","378,366","23,025","91,104","69,694","191,423"
1,2018,종로구,소계,"39,952","13,442","265,017","152,717","112,300",3,17,...,89,"15,658",850,"13,504",744,"15,888",909,"6,227","2,448","7,672"
2,2018,종로구,사직동,"3,541","1,191","49,536","28,658","20,878",1,14,...,19,"6,312",86,967,103,870,95,"1,544",252,843
3,2018,종로구,삼청동,712,324,"4,577","2,193","2,384",-,-,...,4,"1,204",14,260,7,30,17,175,24,209
4,2018,종로구,부암동,565,240,"3,609","1,706","1,903",-,-,...,3,59,40,"1,424",15,148,16,40,83,362


In [190]:
conditions = (sgg_pop2_df['자치구'] != '합계') & (sgg_pop2_df['동'] == '소계')
sgg_biz_df_select = sgg_pop2_df[conditions]
sgg_biz_df_select.head()

,기간,자치구,동,사업체수,여성대표자,계,남,여,사업체수.1,종사자수,...,사업체수.15,종사자수.14,사업체수.16,종사자수.15,사업체수.17,종사자수.16,사업체수.18,종사자수.17,사업체수.19,종사자수.18
1,2018,종로구,소계,"39,952","13,442","265,017","152,717","112,300",3,17,...,89,"15,658",850,"13,504",744,"15,888",909,"6,227","2,448","7,672"
19,2018,중구,소계,"60,957","22,669","390,530","210,755","179,775",-,-,...,62,"19,280",471,"8,321",747,"10,104",558,"3,723","2,138","8,366"
35,2018,용산구,소계,"20,813","7,153","141,216","76,498","64,718",-,-,...,47,"3,149",550,"7,452",579,"6,658",408,"2,831","1,641","5,939"
52,2018,성동구,소계,"27,868","8,432","174,819","100,694","74,125",1,3,...,47,"4,471",961,"11,371",748,"8,539",581,"2,157","2,115","7,612"
70,2018,광진구,소계,"24,535","8,948","127,879","69,442","58,437",1,3,...,36,"2,662","1,183","14,139",909,"11,615","1,033","3,862","2,587","5,608"


In [196]:
columns = ['자치구','계','사업체수']
sgg_biz_df_final = sgg_biz_df_select[columns]
sgg_biz_df_final.columns = ['시군구명','종사자수','사업체수']
sgg_biz_df_final = sgg_biz_df_final.reset_index(drop=True)
sgg_biz_df_final.to_excel('./files/sgg_biz_class.xlsx', index = False)
sgg_biz_df_final.head()

,시군구명,종사자수,사업체수
0,종로구,"265,017","39,952"
1,중구,"390,530","60,957"
2,용산구,"141,216","20,813"
3,성동구,"174,819","27,868"
4,광진구,"127,879","24,535"


### 03 Data Preprocessing

In [15]:
seoul_sb = pd.read_excel('./files/seoul_starbucks_class.xlsx', header = 0)
seoul_sb.head()

,name,lat,long,store_type,address,tel
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
2,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
3,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
4,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232


In [22]:
# seoul_sb['address'][0][6:9]
seoul_sb['address'][0].split()[1]

'강남구'

In [16]:
sigungu_list = [ sigungu[6:9] for sigungu in seoul_sb['address'] ]
seoul_sb['sigungu'] = sigungu_list
seoul_sb.head()

,name,lat,long,store_type,address,tel,sigungu
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
1,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
2,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
3,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
4,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구


In [246]:
seoul_sb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161596 entries, 0 to 161595
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   name        161596 non-null  object 
 1   lat         161596 non-null  float64
 2   long        161596 non-null  float64
 3   store_type  161596 non-null  object 
 4   address     161596 non-null  object 
 5   tel         161596 non-null  object 
 6   sigungu     161596 non-null  object 
dtypes: float64(2), object(5)
memory usage: 8.6+ MB


In [249]:
seoul_sb_sgg_count = seoul_sb.pivot_table(values = 'name',
                                          index = 'sigungu',
                                          aggfunc = 'count')
seoul_sb_sgg_count = seoul_sb_sgg_count.rename(columns = {'name':'starbucks_cnt'})
seoul_sb_sgg_count.head()

,starbucks_cnt
sigungu,
강남구,46156
강동구,1160
강북구,2865
강서구,10197
관악구,4917
